# Import Libraries

In [1]:
import collections
import string
import random
import csv
import sys
import matplotlib.pyplot as plt
import networkx as nx


In [2]:
import glob

In [25]:
from nf1 import NF1

In [49]:
import numpy
from munkres import Munkres

# Load Files

In [3]:
circles_files = glob.glob("twitter/*.circles")
edges_files = glob.glob("twitter/*.edges")
egofeat_files = glob.glob("twitter/*.egofeat")
feat_files = glob.glob("twitter/*.feat")
featnames_files = glob.glob("twitter/*.featnames")

# Create List of egos[973 ego list]


In [4]:
egoNodeList = []
for item in circles_files:
    twitter, circleFilename = item.split("\\")
    filename, abcd = circleFilename.split(".")
    egoNodeList.append(filename)

In [5]:
len(egoNodeList)

973

# Build Graph and create Adjacency Matrix

In [30]:
def load_graph(path,name):
    G = collections.defaultdict(dict)
    graph=nx.DiGraph()

    reader = csv.reader(open(path), delimiter=' ')
    for line in reader:
        v_i = int(line[0])
        v_j = int(line[1])
        G[v_i][v_j] = 1
        G[v_j][v_i] = 1
        graph.add_edge(v_i,v_j,color='b')
    return G

# Read Ground Truth Files

In [31]:
def read_circles(filename):
    final_lst = []
    for line in open(filename):
        lst = line.split('\t')
        el, es = lst[0], lst[1:]
        circle  =set()
        for e in es:
            circle.add(int(e))
        final_lst.append(circle)
    return final_lst

# Define 2 stage  louvain Algo

In [8]:
class Vertex():
    
    def __init__(self, vid, cid, nodes, k_in=0):
        self._vid = vid
        self._cid = cid
        self._nodes = nodes
        self._kin = k_in 

class Louvain():
    
    def __init__(self, G):
        self._G = G
        self._m = 0
        self._cid_vertices = {}
        self._vid_vertex = {}
        for vid in self._G.keys():
            self._cid_vertices[vid] = set([vid])
            self._vid_vertex[vid] = Vertex(vid, vid, set([vid]))
            self._m += sum([1 for neighbor in self._G[vid].keys() if neighbor>vid])
        
    def first_stage(self):
        print ('---first stage---')
        mod_inc = False  
        visit_sequence = self._G.keys()
        random.shuffle(list(visit_sequence))
        while True:
            can_stop = True
            for v_vid in visit_sequence:
                v_cid = self._vid_vertex[v_vid]._cid
                k_v = sum(self._G[v_vid].values()) + self._vid_vertex[v_vid]._kin
                cid_Q = {}
                for w_vid in self._G[v_vid].keys():
                    w_cid = self._vid_vertex[w_vid]._cid
                    if w_cid in cid_Q:
                        continue
                    else:
                        tot = sum([sum(self._G[k].values())+self._vid_vertex[k]._kin for k in self._cid_vertices[w_cid]])
                        if w_cid == v_cid:
                            tot -= k_v
                        k_v_in = sum([v for k,v in self._G[v_vid].items() if k in self._cid_vertices[w_cid]])
                        delta_Q = k_v_in - k_v * tot / self._m
                        cid_Q[w_cid] = delta_Q
                    
                cid,max_delta_Q = sorted(cid_Q.items(),key=lambda item:item[1],reverse=True)[0]
                if max_delta_Q > 0.0 and cid!=v_cid:
                    
                    self._vid_vertex[v_vid]._cid = cid
                    self._cid_vertices[cid].add(v_vid)
                    self._cid_vertices[v_cid].remove(v_vid)
                    can_stop = False
                    mod_inc = True
            if can_stop:
                break
        return mod_inc
        
    def second_stage(self):
        print('---second stage---')
        cid_vertices = {}
        vid_vertex = {}
        for cid,vertices in self._cid_vertices.items():
            if len(vertices) == 0:
                continue
            new_vertex = Vertex(cid, cid, set())
            for vid in vertices:
                new_vertex._nodes.update(self._vid_vertex[vid]._nodes)
                new_vertex._kin += self._vid_vertex[vid]._kin
                for k,v in self._G[vid].items():
                    if k in vertices:
                        new_vertex._kin += v/2.0
            cid_vertices[cid] = set([cid])
            vid_vertex[cid] = new_vertex
        
        G = collections.defaultdict(dict)   
        for cid1,vertices1 in self._cid_vertices.items():
            if len(vertices1) == 0:
                continue
            for cid2,vertices2 in self._cid_vertices.items():
                if cid2<=cid1 or len(vertices2)==0:
                    continue
                edge_weight = 0.0
                for vid in vertices1:
                    for k,v in self._G[vid].items():
                        if k in vertices2:
                            edge_weight += v
                    G[cid1][cid2] = edge_weight
                    G[cid2][cid1] = edge_weight
        
        self._cid_vertices = cid_vertices
        self._vid_vertex = vid_vertex
        self._G = G
    
    def get_communities(self):
        communities = []
        for vertices in self._cid_vertices.values():
            if len(vertices) != 0:
                c = set()
                for vid in vertices:
                    c.update(self._vid_vertex[vid]._nodes)
                communities.append(c)
        return communities
    
    def execute(self):
        iter_time = 1
        while True:
            iter_time += 1
            mod_inc = self.first_stage()
            if mod_inc:
                self.second_stage()
            else:
                break
        return self.get_communities()

# Define Loss Function 

In [17]:

def loss1(usersPerCircle, usersPerCircleP):
    psize = max(len(usersPerCircle),len(usersPerCircleP)) 
    mm = numpy.zeros((psize,psize))
    mm2 = numpy.zeros((psize,psize))
    for i in range(psize):
        for j in range(psize):
            circleP = set() 
            circle = set() 
            if (i < len(usersPerCircleP)):
                circleP = usersPerCircleP[i]
            if (j < len(usersPerCircle)):
                circle = usersPerCircle[j]
            nedits = len(circle.union(circleP)) - len(circle.intersection(circleP)) 
            mm[i][j] = nedits
            mm2[i][j] = nedits

    if psize == 0:
        return 0 
    else:
        m = Munkres()
        indices = m.compute(mm) # Compute the optimal alignment between predicted and groundtruth circles
        editCost = 0
        for row, column in indices:
            editCost += mm2[row][column]
    return int(editCost)

# Calculate Total Loss for all egonet

In [41]:
if __name__ == '__main__':
    totalLoss = 0
    for ego in egoNodeList:
        G = load_graph("twitter/"+ego+".edges",ego)
        algorithm = Louvain(G)
        communities = algorithm.execute()
        gt_circles = read_circles("twitter/"+ego+".circles")
        calLoss = loss1(gt_circles, communities)
        print(calLoss)
        totalLoss +=calLoss
        cnt+=1
        print(cnt)

---first stage---
---second stage---
---first stage---
---second stage---
---first stage---
242
1082
---first stage---
---second stage---
---first stage---
---second stage---
---first stage---
---second stage---
---first stage---
91
1083
---first stage---
---second stage---
---first stage---
---second stage---
---first stage---
---second stage---
---first stage---
19
1084
---first stage---
---second stage---
---first stage---
12
1085
---first stage---
---second stage---
---first stage---
---second stage---
---first stage---
89
1086
---first stage---
---second stage---
---first stage---
---second stage---
---first stage---
---second stage---
---first stage---
221
1087
---first stage---
---second stage---
---first stage---
---second stage---
---first stage---
---second stage---
---first stage---
85
1088
---first stage---
---second stage---
---first stage---
---second stage---
---first stage---
---second stage---
---first stage---
126
1089
---first stage---
---second stage---
---first sta

In [43]:
print(totalLoss)

147356


In [44]:
avgLoss = totalLoss/973

In [45]:
avgLoss

151.44501541623845

## Calculate Normalized F1 for louvian 

In [33]:
if __name__ == '__main__':
    import os
    cnt = 0
    Final_op = {}
    f1_result_map ={}
    f1_result_val = []
    for ego in egoNodeList:
        G = load_graph("twitter/"+ego+".edges",ego)
        algorithm = Louvain(G)
        communities = algorithm.execute()

        gt_circles = read_circles("twitter/"+ego+".circles")

        if communities and gt_circles:
            nf = NF1(communities,gt_circles)
            results = nf.summary()
            df = results['scores']
            nf1 = df["Value"][5]
            f1_result_map[ego] = nf1
            f1_result_val += nf1
        cnt+=1
        print(cnt)

---first stage---
---second stage---
---first stage---
---second stage---
---first stage---
1
---first stage---
---second stage---
---first stage---
---second stage---
---first stage---
---second stage---
---first stage---
2
---first stage---
---second stage---
---first stage---
---second stage---
---first stage---
---second stage---
---first stage---
3
---first stage---
---second stage---
---first stage---
4
---first stage---
---second stage---
---first stage---
---second stage---
---first stage---
5
---first stage---
---second stage---
---first stage---
---second stage---
---first stage---
---second stage---
---first stage---
6
---first stage---
---second stage---
---first stage---
---second stage---
---first stage---
---second stage---
---first stage---
7
---first stage---
---second stage---
---first stage---
---second stage---
---first stage---
---second stage---
---first stage---
8
---first stage---
---second stage---
---first stage---
---second stage---
---first stage---
---secon

In [51]:
# f1_result_map

# Avg of Nomralized F1

In [36]:
AvgF1 = sum(f1_result_map.values())/973

In [37]:
AvgF1

0.05769619755494924